In [ ]:
import tensorflow as tf
import numpy as np

from tensorflow.examples.tutorials.mnist import input_data

train_dir = "E:\\home\\workspace\\datasets\\mnist"
mnist = input_data.read_data_sets(train_dir=train_dir,one_hot=True)

In [ ]:
def inference(inputs,training=True):
    outputs = tf.reshape(inputs,[-1,28,28,1])
    outputs = tf.layers.conv2d(outputs,filters=6,kernel_size=3)
    outputs = tf.layers.batch_normalization(outputs,training=training)
    outputs = tf.nn.relu(outputs)
    
    outputs = tf.layers.conv2d(outputs,filters=16,kernel_size=3)
    outputs = tf.layers.batch_normalization(outputs,training=training)
    outputs = tf.nn.relu(outputs)
    
    outputs = tf.layers.conv2d(outputs,filters=64,kernel_size=3)
    outputs = tf.layers.batch_normalization(outputs,training=training)
    outputs = tf.nn.relu(outputs)
    
    outputs = tf.layers.flatten(outputs)
    
    outputs = tf.layers.dense(outputs,84,activation=tf.nn.relu)
    
    outputs = tf.layers.dense(outputs,10,activation=tf.nn.softmax)
    
    return outputs    

In [ ]:
x = tf.placeholder(tf.float32,shape=[None,784])
y = tf.placeholder(tf.float32,(None,10))

y_pred = inference(x)



In [ ]:
var = tf.trainable_variables()
regularizer = tf.contrib.layers.l2_regularizer(scale=.0001)
l2_var = tf.contrib.layers.apply_regularization(regularizer,var)

loss = tf.reduce_mean(tf.reduce_sum(-y*tf.log(y_pred),reduction_indices=[1])) + l2_var

In [ ]:
training = False
if training:
    update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
    with tf.control_dependencies(update_ops):
        optimizer = tf.train.AdamOptimizer().minimize(loss)
else:
    optimizer = tf.train.AdamOptimizer().minimize(loss)


In [ ]:
accuracy = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(y_pred,axis=1),tf.argmax(y,axis=1)),tf.float32))

In [ ]:
var_list = tf.trainable_variables()
print(len(var_list))
g_list = tf.global_variables()
bn_moving_vars = [g for g in g_list if 'moving_mean' in g.name]
bn_moving_vars += [g for g in g_list if 'moving_variance' in g.name]
var_list += bn_moving_vars
print(len(var_list))

In [ ]:
model_path = "./Model_save/model_cnn.ckpt"
saver = tf.train.Saver(var_list=var_list, max_to_keep=5)

In [ ]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    try:
        saver.restore(sess,save_path=model_path)
        print("last model and prams restore path:",model_path)
     
    except Exception as e:
        print(e.message())
        print("not found restore path")
        
    finally:
        for it in range(11):
            xs,ys = mnist.train.next_batch(128)
            sess.run(optimizer,feed_dict={x:xs,y:ys})

            if it % 10 == 0:
                xs,ys = mnist.test.next_batch(3000)
                v_loss, v_accuracy = sess.run([loss,accuracy],{x:xs,y:ys})
                print("> Iter num: %d, Test loss is %.2f, accuracy is %.4f"%(it+1,v_loss,v_accuracy))

        saver.save(sess=sess,save_path=model_path)
        print("latest model and prams save path:",model_path)